In [1]:
# 시도명 매핑 딕셔너리
sido_map = {
    '서울': '서울특별시', '부산': '부산광역시', '대구': '대구광역시',
    '광주': '광주광역시', '울산': '울산광역시', '인천': '인천광역시',
    '대전': '대전광역시',
    
    # 도 단위 및 기타 지역 매핑
    '북춘천': '강원도', '북강릉': '강원도', '울릉도': '경상북도',
    '안동': '경상북도', '포항': '경상북도', '창원': '경상남도',
    '수원': '경기도', '청주': '충청북도', '전주': '전라북도',
    '목포': '전라남도', '여수': '전라남도', '흑산도': '전라남도',
    '홍성': '충청남도', '백령도': '인천광역시', '서귀포': '제주도',
    '제주': '제주도',
    
    # 기타 추가
    '속초': '강원도', '철원': '강원도', '동두천': '경기도', '파주': '경기도',
    '대관령': '강원도', '춘천': '강원도', '강릉': '강원도', '동해': '강원도',
    '원주': '강원도', '영월': '강원도', '충주': '충청북도', '서산': '충청남도',
    '울진': '경상북도', '추풍령': '경상북도', '상주': '경상북도',
    '군산': '전라북도', '통영': '경상남도', '완도': '전라남도', '고창': '전라북도',
    '순천': '전라남도', '진도(첨찰산)': '전라남도', '서청주': '충청북도',
    '고산': '제주도', '성산': '제주도', '진주': '경상남도',
    '강화': '인천광역시', '양평': '경기도', '이천': '경기도',
    '인제': '강원도', '홍천': '강원도', '태백': '강원도', '정선군': '강원도',
    '제천': '충청북도', '보은': '충청북도', '천안': '충청남도',
    '보령': '충청남도', '부여': '충청남도', '금산': '충청남도',
    '세종': '세종특별자치시', '부안': '전라북도', '임실': '전라북도',
    '정읍': '전라북도', '남원': '전라북도', '장수': '전라북도',
    '고창군': '전라북도', '영광군': '전라남도', '김해시': '경상남도',
    '순창군': '전라북도', '북창원': '경상남도', '양산시': '경상남도',
    '보성군': '전라남도', '강진군': '전라남도', '장흥': '전라남도',
    '해남': '전라남도', '고흥': '전라남도', '의령군': '경상남도',
    '함양군': '경상남도', '광양시': '전라남도', '진도군': '전라남도',
    '봉화': '경상북도', '영주': '경상북도', '문경': '경상북도',
    '청송군': '경상북도', '영덕': '경상북도', '의성': '경상북도',
    '구미': '경상북도', '영천': '경상북도', '경주시': '경상북도',
    '거창': '경상남도', '합천': '경상남도', '밀양': '경상남도',
    '산청': '경상남도', '거제': '경상남도', '남해': '경상남도',
    '북부산': '부산광역시'
}

In [7]:
import pandas as pd

# 로드
df_temp = pd.read_csv('./data/OBS_ASOS_DD_20251124115512.csv', encoding='cp949')

# 시도명 매핑
df_temp['시도명'] = df_temp['지점명'].map(sido_map)

# 33도 이상 필터링
df_temp = df_temp[df_temp['최고기온(°C)'] >= 33]

# 날짜 필터링 (수확기) - 6월 20일 ~ 9월 10일로 잡음
df_temp['일시'] = pd.to_datetime(df_temp['일시'], format='%Y-%m-%d', errors='coerce')
start_date = pd.to_datetime('2000-06-20')
end_date = pd.to_datetime('2000-09-10')
df_temp = df_temp[
    df_temp['일시'].apply( lambda x: start_date.replace(year=x.year) <= x <= end_date.replace(year=x.year) )
]

# 연도별, 시도명별 폭염 일수 집계
df_temp['year'] = df_temp['일시'].dt.year
heatwave_days = df_temp.groupby(['year', '시도명'])['일시'].count().reset_index(name='폭염일수_관측소합계')
station_counts = df_temp.groupby(['year', '시도명'])['지점명'].nunique().reset_index(name='관측소수')
heatwave_days = pd.merge(heatwave_days, station_counts, on=['year', '시도명'])
heatwave_days['폭염일수 평균'] = (heatwave_days['폭염일수_관측소합계'] / heatwave_days['관측소수']).round(2)
heatwave_days

,year,시도명,폭염일수_관측소합계,관측소수,폭염일수 평균
0,2018,강원도,321,13,24.69
1,2018,경기도,174,5,34.80
2,2018,경상남도,504,14,36.00
3,2018,경상북도,463,15,30.87
4,2018,광주광역시,43,1,43.00
...,...,...,...,...,...
113,2024,전라남도,331,13,25.46
114,2024,전라북도,269,10,26.90
115,2024,제주도,75,4,18.75
116,2024,충청남도,142,6,23.67


In [ ]:
heatwave_days.drop(['폭염일수_관측소합계', '관측소수'], axis=1, inplace=True)

# 세종특별자치시 2018년 데이터 또 누락이라 추가
sejong_2018 = {
    'year': 2018,
    '시도명': '세종특별자치시',
    '폭염일수 평균': heatwave_days[
        (heatwave_days['year'] == 2018) & (heatwave_days['시도명'] == '대전광역시')
    ]['폭염일수 평균'].values[0]
}

heatwave_days = pd.concat([heatwave_days, pd.DataFrame([sejong_2018])], ignore_index=True)
heatwave_days = heatwave_days.sort_values(by=['year', '시도명']).reset_index(drop=True)

heatwave_days.to_csv('./data/시도별_폭염일수_평균.csv', index=False, encoding='utf-8')